In [1]:
import os
import nltk
import pandas
import collections
import itertools
import numpy as np
import math
import string

In [2]:
K = 0.1
POS_PATH = './posTrain/'
NEG_PATH = './negTrain/'
POS_TEST_PATH = './posTest/'
NEG_TEST_PATH = './negTest/'

In [3]:
# given to us to remove punctuation and capital letters
tokenizer = nltk.tokenize.RegexpTokenizer(r'[a-zA-Z]+[\']*[a-zA-Z]+|[;!?$]')

In [4]:
# Get all the training files.. this will take a sec
posFileList = [POS_PATH+f for f in os.listdir(POS_PATH)]
negFileList = [NEG_PATH+f for f in os.listdir(NEG_PATH)]

In [5]:
# clean pos file list to clean text and write to word map to keep track of the frequency of words across
# all pos documents. Also keep track of how many pos files there are and how many pos words in total
N_POS_WORDS = 0;
word_map_pos = {}
for file in posFileList:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        rawText = f.read()
        cleanText = tokenizer.tokenize(rawText)
        # I went back and forth with this initilly but we need to grab all the words that 
        # will be used later to calculate probability 
        N_POS_WORDS += len(cleanText)
        for word in cleanText:
            if (word in word_map_pos.keys()):
                word_map_pos[word] += 1
            else:
                word_map_pos[word] = 1

# clean neg file list to clean text and write to word map to keep track of the frequency of words across
# all neg documents. Also keep track of how many neg files there are and how many neg words in total
N_NEG_WORDS = 0;
word_map_neg = {}
for file in negFileList:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        rawText = f.read()
        cleanText = tokenizer.tokenize(rawText)
        # I went back and forth with this initilly but we need to grab all the words that 
        # will be used later to calculate probability 
        N_NEG_WORDS += len(cleanText)
        for word in cleanText:
            if (word in word_map_neg.keys()):
                word_map_neg[word] += 1
            else:
                word_map_neg[word] = 1

In [6]:
# calculate probability of pos w/o smoothing yet
# 
pos_prob = 0.0;
pos_prob_word_map = {}
for word in word_map_pos:
    #pos_prob += math.log(word_map_pos.get(word)/N_POS_WORDS)
    pos_prob_word_map[word] = (word_map_pos.get(word)/N_POS_WORDS)

# calculate probability of neg w/o smoothing yet
neg_prob = 0.0;
neg_prob_word_map = {}
for word in word_map_neg:
    #neg_prob += math.log(word_map_neg.get(word)/N_NEG_WORDS)
    neg_prob_word_map[word] = (word_map_neg.get(word)/N_NEG_WORDS)


In [7]:
posTestList = [POS_TEST_PATH+f for f in os.listdir(POS_TEST_PATH)]
negTestList = [NEG_TEST_PATH+f for f in os.listdir(NEG_TEST_PATH)]

In [8]:
conditionalProbPos = {}
conditionalProbNeg = {}
gamma = 0.1
# Calculate smoothing and prior

In [10]:
# calculate unique words dictionary that aren't in both positive or negative
unique_pos_words = []
for w in word_map_pos:
    if (w not in word_map_neg):
        unique_pos_words.append(w)

unique_neg_words = []
for w in word_map_neg:
    if (w not in word_map_pos):
        unique_neg_words.append(w)

# calculate the variable d that will be used in the smoothing equation
d = len(unique_neg_words) + len(unique_pos_words)

In [ ]:
# calculate unique words dictionary that aren't in both positive or negative
for w in word_map_pos:
    if (w not in word_map_neg):
        # we need to set the coniditonal propbability in the negative file to not be 0 but 
        # will be the smoothing equation with 0 frequency for that word b/c we still need
        # to account for the word
        conditionalProbNeg[w] = ((0 + gamma) / (N_NEG_WORDS + (gamma*d)))

for w in word_map_neg:
    if (w not in word_map_pos):
        # we need to set the conditional probability in the postive file to not be 0 but
        # will be the smoothing equation with 0 frequency for that word b/c we still need
        # to account for the word
        conditionalProbPos[w] = ((0 + gamma) / (N_POS_WORDS + (gamma*d)))

In [ ]:
# calculate probability of pos with smoothing now
for word in word_map_pos:
    #pos_prob_smooth += math.log(((word_map_pos.get(word) + gamma) / (N_POS_WORDS + (gamma*d))))
    conditionalProbPos[word] = ((word_map_pos.get(word) + gamma) / (N_POS_WORDS + (gamma*d)))
    

# calculate probability of neg with smoothing now
for word in word_map_neg:
    #neg_prob_smooth += math.log((word_map_neg.get(word) +gamma) / (N_NEG_WORDS + (gamma*d)))
    conditionalProbNeg[word] = ((word_map_neg.get(word) + gamma) / (N_NEG_WORDS + (gamma*d)))

In [11]:
# compute pos test results
resultsP = {}
prob_neg = 0
prob_pos = 0

for file in posTestList:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        rawText = f.read()
        cleanText = tokenizer.tokenize(rawText)
        for word in cleanText:
            # check if word is in our trained word map. If it isn't skip it 
            # b/c we aren't totally sure the probability of it being pos or neg
            if (word in conditionalProbPos.keys()):
                prob_pos += math.log(conditionalProbPos.get(word))
                
            if (word in conditionalProbNeg.keys()):
                prob_neg += math.log(conditionalProbNeg.get(word))
                
        # compare which probability is bigger. So, in this case if positive probability is larger
        # then we have successfully classified this particular file
        if (prob_pos > prob_neg):
            resultsP[file] = 1
        else:
            resultsP[file] = 0
    prob_pos = 0
    prob_neg = 0

# compute neg test results
resultsN = {}
for file in negTestList:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        rawText = f.read()
        cleanText = tokenizer.tokenize(rawText)
        for word in cleanText:
            # check if word is in our trained word map. If it isn't skip it 
            # b/c we aren't totally sure the probability of it being pos or neg
            if (word in conditionalProbPos.keys()):
                prob_pos += math.log(conditionalProbPos.get(word))
                
            if (word in conditionalProbNeg.keys()):
                prob_neg += math.log(conditionalProbNeg.get(word))
                
        # compare which probability is bigger. So, in this case if negative probability is larger
        # then we have successfully classified this particular file
        if (prob_neg > prob_pos):
            resultsN[file] = 1
        else:
            resultsN[file] = 0
    prob_pos = 0
    prob_neg = 0


In [12]:
# given to us but essentially it checks how we did building a confusion matrix determining
# the true positives (classified right), false positives (classified wrong), true negative (classified right)
# and false negative (classified wrong)
TP = sum(resultsP.values())
FN = len(resultsP.values()) - TP
TN = sum(resultsN.values())
FP = len(resultsN.values()) - TN
acc = (TP+TN)/(TP+FN+FP+TN)
print('Pred \ Gold \t P \t N')
print('P \t \t '+str(TP)+' \t '+str(FP))
print('N \t \t '+str(FN)+' \t '+str(TN))

Pred \ Gold 	 P 	 N
P 	 	 73 	 21
N 	 	 16 	 69


In [13]:
# given but calculates accuracy with number of successfully guessed files over total classification
acc = (TP+TN)/float(TP+FN+FP+TN)
print('accuracy is: ')
print(acc)

accuracy is: 
0.7932960893854749
